In [1]:
!git config --global user.email "ekiprop26@gmail.com"
!git config --global user.name "ekiprop"

!pip install --upgrade pip
!pip install transformers wandb peft accelerate trl bitsandbytes datasets huggingface-hub safetensors pandas matplotlib numpy scipy scikit-learn
!pip install summa evaluate rouge_score pyarrow sentencepiece
!pip install watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB

In [8]:
!wget https://raw.githubusercontent.com/Nicolepcx/Transformers-in-Action/refs/heads/main/utils/utils.py
!wget https://raw.githubusercontent.com/Nicolepcx/Transformers-in-Action/refs/heads/main/utils/textrank.py
from utils import *
from textrank import *

--2025-03-14 08:54:16--  https://raw.githubusercontent.com/Nicolepcx/Transformers-in-Action/refs/heads/main/utils/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18698 (18K) [text/plain]
Saving to: ‘utils.py.1’

utils.py.1          100%[===================>]  18.26K  --.-KB/s    in 0s      

2025-03-14 08:54:16 (112 MB/s) - ‘utils.py.1’ saved [18698/18698]

--2025-03-14 08:54:16--  https://raw.githubusercontent.com/Nicolepcx/Transformers-in-Action/refs/heads/main/utils/textrank.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
L

In [9]:
import pandas as pd
import numpy as np
from functools import partial
import logging

import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from summa import summarizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, GenerationConfig, set_seed
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
from transformers import LEDForConditionalGeneration, LEDTokenizer
import torch
from datasets import load_dataset

from multiprocessing import Pool
import evaluate
from tqdm import tqdm

import matplotlib.pyplot as plt

from textwrap import TextWrapper

set_seed(42)

In [10]:
## Loading the BillSum dataset
dataset = load_dataset("billsum")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})

In [12]:
dataset_test =load_dataset("billsum", split="ca_test")
dataset_test.set_format(type="pandas", columns=["text", "summary"])

In [13]:
df = dataset_test.to_pandas()

In [15]:
df.head()

,text,summary,title
0,The people of the State of California do enact...,Existing property tax law establishes a vetera...,An act to amend Section 215.1 of the Revenue a...
1,The people of the State of California do enact...,Existing law provides that the Board of Parole...,"An act to amend Section 3550 of, and to add Se..."
2,The people of the State of California do enact...,The Sales and Use Tax Law imposes a tax on ret...,An act\nto add Chapter 3.8 (commencing with Se...
3,The people of the State of California do enact...,"Existing law requires all moneys, except for f...","An act to amend Sections 75220, 75221, and 752..."
4,The people of the State of California do enact...,Existing law defines a request regarding resus...,An act to add and repeal Section 4788 of the P...


In [16]:
## Excerpt of article and ground truth of summary
sample = df.iloc[0]

excerpt = 1000

print(f"\033[1mExcerpt of {excerpt} characters, total length of article: \
{len(sample['text'])}:\033[0m\n")

print(sample["text"][:excerpt])
print(f"\033[1m\n\nSummary (length: {len(sample['summary'])}):\033[0m\n")
print(sample["summary"])

Excerpt of 1000 characters, total length of article: 8203:

The people of the State of California do enact as follows:


SECTION 1.
The Legislature finds and declares all of the following:
(a) (1) Since 1899 congressionally chartered veterans’ organizations
have provided a valuable service to our nation’s returning service
members. These organizations help preserve the memories and incidents
of the great hostilities fought by our nation, and preserve and
strengthen comradeship among members.
(2) These veterans’ organizations also own and manage various
properties including lodges, posts, and fraternal halls. These
properties act as a safe haven where veterans of all ages and their
families can gather together to find camaraderie and fellowship, share
stories, and seek support from people who understand their unique
experiences. This aids in the healing process for these returning
veterans, and ensures their health and happiness.
(b) As a result of congressional chartering of these vete

In [17]:
articles_np = np.array(dataset_test['text'])
articles_np.shape

(1237,)

In [18]:
articles_np_1 = articles_np[:1]
articles_np_1.shape

(1,)

In [19]:
print(articles_np_1)

['The people of the State of California do enact as
follows:\n\n\nSECTION 1.\nThe Legislature finds and declares all of
the following:\n(a) (1) Since 1899 congressionally chartered veterans’
organizations have provided a valuable service to our nation’s
returning service members. These organizations help preserve the
memories and incidents of the great hostilities fought by our nation,
and preserve and strengthen comradeship among members.\n(2) These
veterans’ organizations also own and manage various properties
including lodges, posts, and fraternal halls. These properties act as
a safe haven where veterans of all ages and their families can gather
together to find camaraderie and fellowship, share stories, and seek
support from people who understand their unique experiences. This aids
in the healing process for these returning veterans, and ensures their
health and happiness.\n(b) As a result of congressional chartering of
these veterans’ organizations, the United States Internal Rev

In [20]:
## Applying TextRank algorithms to BillSum dataset
# Define the function to summarize the text
def summarize_text(text, words=250):
  summary = summarizer.summarize(text, words=words)
  return summary

# Parallelize the TextRank summarization
with Pool() as pool:
  summarized_articles = pool.map(summarize_text, articles_np_1)

In [21]:
print(summarized_articles)

['(c) Section 501(c)(19) of the Internal Revenue Code and related
federal regulations provide for the exemption for posts or
organizations of war veterans, or an auxiliary unit or society of, or
a trust or foundation for, any such post or organization that, among
other attributes, carries on programs to perpetuate the memory of
deceased veterans and members of the Armed Forces and to comfort their
survivors, conducts programs for religious, charitable, scientific,
literary, or educational purposes, sponsors or participates in
activities of a patriotic nature, and provides social and recreational
activities for their members.\n(a) All buildings, and so much of the
real property on which the buildings are situated as may be required
for the convenient use and occupation of the buildings, used
exclusively for charitable purposes, owned by a veterans’ organization
that has been chartered by the Congress of the United States,
organized and operated for charitable purposes, and exempt from
f

In [22]:
references = np.array(dataset_test['summary'])

In [23]:
references_np  = np.array(references)
print(references_np[:1])

['Existing property tax law establishes a veterans’ organization
exemption under which property is exempt from taxation if, among other
things, that property is used exclusively for charitable purposes and
is owned by a veterans’ organization.\nThis bill would provide that
the veterans’ organization exemption shall not be denied to a property
on the basis that the property is used for fraternal, lodge, or social
club purposes, and would make specific findings and declarations in
that regard. The bill would also provide that the exemption shall not
apply to any portion of a property that consists of a bar where
alcoholic beverages are served.\nSection 2229 of the Revenue and
Taxation Code requires the Legislature to reimburse local agencies
annually for certain property tax revenues lost as a result of any
exemption or classification of property for purposes of ad valorem
property taxation.\nThis bill would provide that, notwithstanding
Section 2229 of the Revenue and Taxation Code, no 

In [24]:
reference = references[0]

In [25]:
summaries = {}
summaries['TextRank (Baseline)'] = summarized_articles
print(summaries["TextRank (Baseline)"])

['(c) Section 501(c)(19) of the Internal Revenue Code and related
federal regulations provide for the exemption for posts or
organizations of war veterans, or an auxiliary unit or society of, or
a trust or foundation for, any such post or organization that, among
other attributes, carries on programs to perpetuate the memory of
deceased veterans and members of the Armed Forces and to comfort their
survivors, conducts programs for religious, charitable, scientific,
literary, or educational purposes, sponsors or participates in
activities of a patriotic nature, and provides social and recreational
activities for their members.\n(a) All buildings, and so much of the
real property on which the buildings are situated as may be required
for the convenient use and occupation of the buildings, used
exclusively for charitable purposes, owned by a veterans’ organization
that has been chartered by the Congress of the United States,
organized and operated for charitable purposes, and exempt from
f

In [1]:
def generate_summary(model_name, sample_text):
  # Set up tokenizer and model
  model_dict = {
      "BART": "facebook/bart-large-cnn",
      "T5": "sysresearch101/t5-large-finetuned-xsum-cnn",
      "ProphetNet": "microsoft/prophetnet-large-uncased-cnndm"
  }

  if model_name == "Pegasus":
    # Generate summary using the specified model
    summarization_pipeline = pipeline("summarization", model="google/pegasus-cnn_dailymail")
    chunks  = [sample_text[i:i+2048] for i in range(0, len(sample_text), 2048)]
    summaries = []

    for chunk in chunks:
      summarization_output = summarization_pipeline(chunk, max_length=512)
      summaries.append(summarization_output[0]["summary_text"].replace(" .", ".\n"))
    summary = " ".join(summaries)
  elif model_name == in model_dict.keys():
    tokenizer = AutoTokenizer.from_pretrained(model_dict[model_name], max_length=1024, truncation=True)
    model = AutoModelforSeq2SeqLM.from_pretrained(model_dict[model_name])
    gen_config  = GenerationConfig(max_length = 512)
    summarization_pipeline = pipeline("summarization", tokenizer=tokenizer, model=model, config=gen_config)

    # Generate sumamry using the specified model
    chunks = [sample_text[i:i+1024] for i in range(0, len(sample_text), 1024)]
    summaries = []
    for chunks in chunks:
      summarization_output = summarization_pipeline(chunk)
      summaries.append(summarization_output[0]["summary_text"])
    summary = " ".join(summaries)
  else:
    raise ValueError(f"Model {model_name} is not supported.")
  return summary


SyntaxError: invalid syntax (<ipython-input-1-609abf10e834>, line 19)